In [11]:
# @Date    : Nov-08-20 00:29
# @Author  : Kelly Hwong (dianhuangkan@gmail.com)
# @RefLink : https://keras.io/api/applications/

In [1]:
from datetime import datetime
model_type = "keras.applications.ResNet50V2"

# data path
competition_name = "dogs-vs-cats-redux-kernels-edition"
data_dir = os.path.expanduser(
    f"~/.kaggle/competitions/{competition_name}")

# experiment time
date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
ckpt_dir = os.path.expanduser(
    f"~/Documents/DeepLearningData/{competition_name}/ckpts/{model_type}/{date_time}")
log_dir = os.path.expanduser(
    f"~/Documents/DeepLearningData/{competition_name}/logs/{model_type}/{date_time}")

In [28]:
log_dir = os.path.expanduser(os.path.join("~", "Documents", "DeepLearningData", competition_name, "logs", model_type, date_time))

'C:\\Users\\kellyhwong\\Documents\\DeepLearningData\\dogs-vs-cats-redux-kernels-edition\\logs\\keras.applications.ResNet50V2\\20201113-162840'

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50, ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from utils.data_utils import data_generators

In [13]:
# data path
competition_name = "dogs-vs-cats-redux-kernels-edition"
data_dir = os.path.expanduser(f"~/.kaggle/competitions/{competition_name}")

# Input parameters
IMAGE_WIDTH = IMAGE_HEIGHT = 128
image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
num_classes = 2

# tmp
batch_size = 32

## Data loader

In [14]:
train_generator, validation_generator = data_generators(
        data_dir, target_size=image_size, batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [15]:
train_generator.class_indices

{'cat': 0, 'dog': 1}

In [16]:
input_ = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(input_, tf.float32)
x = preprocess_input(x)
extractor = ResNet50V2(include_top=False, weights='imagenet', input_shape=input_shape, classes=num_classes)
x = extractor(x)
classifier = keras.Sequential([
    keras.Input((4, 4, 2048)),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(num_classes, activation='softmax')], 
    name="classifier")
x = classifier(x)
model = tf.keras.Model(inputs=[input_], outputs=[x], name="model")
# model = keras.Sequential([extractor, classifier], name="model")

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
tf_op_layer_Cast (TensorFlow [(None, None, None, 3)]   0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, None, None, 3)]   0         
_________________________________________________________________
tf_op_layer_BiasAdd (TensorF [(None, None, None, 3)]   0         
_________________________________________________________________
resnet50v2 (Model)           (None, 4, 4, 2048)        23564800  
_________________________________________________________________
classifier (Sequential)      (None, 2)                 4098      
Total params: 23,568,898
Trainable params: 23,523,458
Non-trainable params: 45,440
____________________________________________

In [18]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Recall, Precision, TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, AUC
metrics = [
    Recall(name='recall'),
    Precision(name='precision'),
    TruePositives(name='tp'),  # thresholds=0.5
    FalsePositives(name='fp'),
    TrueNegatives(name='tn'),
    FalseNegatives(name='fn'),
    BinaryAccuracy(name='accuracy'),
    # AUC0(name='auc_cat_0'),  # 以 cat 为 positive 的 AUC
    AUC(name='auc_dog_1')  # 以 dog 为 positive 的 AUC
]
model.compile(loss=BinaryCrossentropy(),
                  optimizer=Adam(learning_rate=1e-3),
                  metrics=metrics)

In [19]:
epochs = 3
history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        # callbacks=callbacks,
        verbose=1
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 625 steps, validate for 157 steps
Epoch 1/3
200/625 [========>.....................] - ETA: 4:08 - loss: 0.5228 - recall: 0.7538 - precision: 0.7538 - tp: 4800.0000 - fp: 1568.0000 - tn: 4800.0000 - fn: 1568.0000 - accuracy: 0.7538 - auc_bad_1: 0.8331

KeyboardInterrupt: 